## This notebook is used for testing MTAOS communication with the AOS components: M1M3, M2, and hexapods

Step-by-step description of what we do can be found in 
Jira test case: https://jira.lsstcorp.org/secure/Tests.jspa#/testCase/LVV-T???? 

test on NTS first:

* Do all the callback messages look right?
* Look at the EFD notebooks at the same time. Is everything as expected?

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import rubin_jupyter_utils.lab.notebook as nb
nb.utils.get_node()

In [ ]:
from lsst.ts import salobj
import asyncio
import os

import numpy as np
from matplotlib import pyplot as plt
from astropy.time import Time
from datetime import datetime, timedelta
import pandas as pd
import yaml

from lsst.ts.observatory.control.maintel.mtcs import MTCS, MTCSUsages
from lsst.ts.observatory.control import RotType
from astropy.coordinates import AltAz, ICRS, EarthLocation, Angle, FK5
import astropy.units as u

from aosTools import *

plt.jet();

In [ ]:
print(os.environ["OSPL_URI"])
print(os.environ["LSST_DDS_PARTITION_PREFIX"])
print(os.environ["LSST_DDS_DOMAIN_ID"])

In [ ]:
#index is an integter which helps avoid multple users starting same controller
script = salobj.Controller("Script", index=7)

In [ ]:
mtcs = MTCS(script.domain)
mtcs.set_rem_loglevel(40)

In [ ]:
await mtcs.start_task

In [ ]:
mtcs.components_attr

In [ ]:
camhex = mtcs.rem.mthexapod_1
m2hex = mtcs.rem.mthexapod_2
m1m3 = mtcs.rem.mtm1m3
m2 = mtcs.rem.mtm2
aos = mtcs.rem.mtaos
mount = mtcs.rem.mtmount
rot = mtcs.rem.mtrotator

In [ ]:
def printLogMessage(data):
    print(f"{data.level}: {data.message}")
aos.evt_logMessage.callback = printLogMessage  #None
await aos.cmd_setLogLevel.set_start(level=10, timeout=5)

In [ ]:
await aos.evt_heartbeat.next(flush=True, timeout=5)

In [ ]:
await salobj.set_summary_state(aos, salobj.State.ENABLED) #, settingsToApply='default') #leave this out!!
#two levels of defaults: csc defaults, config repo; right now csc defaults, they will move to config repo; 
#but to load defaults from config repo, we do not need to supply settingsToApply

In [ ]:
#summit = 1 #use this for summit testing
summit = 0 #use this for NCSA

In [ ]:
from lsst_efd_client import EfdClient

if summit:
    client = EfdClient('summit_efd')
else:
    client = EfdClient('ncsa_teststand_efd')
csc_index = 1

#### Check summary state of each CSC

In [ ]:
await checkAOSSummaryStates(aos, m1m3, m2, camhex, m2hex)

### Get M1M3 Ready: raise mirorr, turn on FB, clear forces

In [ ]:
await m1m3.evt_heartbeat.next(flush=True, timeout=5)

In [ ]:
print('Re-enabling M1M3')
#await salobj.set_summary_state(m1m3, salobj.State.STANDBY) 
await salobj.set_summary_state(m1m3, salobj.State.ENABLED, settingsToApply = 'Default') #enable m1m3

In [ ]:
await readyM1M3(m1m3)

### Get M2 Ready: turn on FB, clear forces

In [ ]:
await m2.evt_heartbeat.next(flush=True, timeout=5)

In [ ]:
await salobj.set_summary_state(m2, salobj.State.ENABLED) 
#await salobj.set_summary_state(m1m3, salobj.State.ENABLED, settingsToApply = 'Default') #enable m1m3

In [ ]:
await readyM2(m2)

### Get cam hex Ready: check config; make sure LUT is on, and has valid inputs; make sure hex is at LUT position

In [ ]:
await camhex.evt_heartbeat.next(flush=True, timeout=5)

In [ ]:
await mount.cmd_moveToTarget.set_start(elevation=75, azimuth=0)

In [ ]:
await rot.cmd_move.set_start(position=0)

In [ ]:
#await camhex.cmd_move.set_start(x=0,y=0,z=100, u=0,v=0,w=0,sync=True)

In [ ]:
await readyHexaForAOS(camhex)

In [ ]:
await camhex.cmd_move.set_start(x=0,y=0,z=0, u=0,v=0,w=0,sync=True)

In [ ]:
await salobj.set_summary_state(camhex, salobj.State.STANDBY)

In [ ]:
await salobj.set_summary_state(camhex, salobj.State.ENABLED, settingsToApply="default")

In [ ]:
state = await camhex.evt_summaryState.aget(timeout=5)
print('staring with: cam hex state', salobj.State(state.summaryState), pd.to_datetime(state.private_sndStamp, unit='s'))
dstate = await camhex.evt_controllerState.aget(timeout=5)
print('          cam hex state', MTHexapod.EnabledSubstate(dstate.enabledSubstate), 
  pd.to_datetime(dstate.private_sndStamp, unit='s'))

In [ ]:
mount_angle = await mount.tel_elevation.next(flush=False, timeout=10.)
print("mount elevation angle", mount_angle.actualPosition)
elev = mount_angle.actualPosition

### Get M2 hex Ready: check config; make sure LUT is on, and has valid inputs; make sure hex is at LUT position

In [ ]:
await m2hex.evt_heartbeat.next(flush=True, timeout=5)

In [ ]:
#await m2hex.cmd_move.set_start(x=0,y=0,z=100, u=0,v=0,w=0,sync=True)

In [ ]:
await readyHexaForAOS(m2hex)

In [ ]:
await salobj.set_summary_state(m2hex, salobj.State.ENABLED, settingsToApply="default")

In [ ]:
await aos.cmd_resetCorrection.set_start()
await aos.cmd_issueCorrection.set_start() 

### Step 1: add aberrations via OFC

In [ ]:
zernikes = np.zeros(19)
zerIdx = np.arange(4, 23)
zernikes[7-4] = 1 #add 1um of z7
print(zerIdx)
print(zernikes)

In [ ]:
#x0 makes sure corrections double when same aberrations are added twice
config = {"xref": "x0"}  
config_yaml = yaml.safe_dump(config)

In [ ]:
await aos.cmd_addAberration.set_start(wf = zernikes, config=config_yaml)

In [ ]:
ofc_dict =  await ofcSentApplied(aos, m1m3, m2, camhex, m2hex, make_plot = True)

In [ ]:
await aos.cmd_issueCorrection.set_start() 

In [ ]:
ofc_dict =  await ofcSentApplied(aos, m1m3, m2, camhex, m2hex, make_plot = True)

#### Look at telemetry to verify the changes in the DOFs

In [ ]:
end = Time(datetime.now(), scale='tai')
#end = Time('2021-05-14T18:25:30', scale = 'tai')
start = end - timedelta(seconds=100)

dfm = await client.select_time_series('lsst.sal.MTMount.elevation', '*', start, end, csc_index)
dfm1m3 = await client.select_time_series('lsst.sal.MTM1M3.logevent_appliedActiveOpticForces', '*', start, end, csc_index)
dfm2 = await client.select_time_series('lsst.sal.MTM2.axialForce', '*', start, end, csc_index)
dfh = await client.select_time_series('lsst.sal.MTHexapod.logevent_uncompensatedPosition', '*', start, end, csc_index)

idx1=dfh.MTHexapodID==1
dfh1 = dfh[idx1]
idx2=dfh.MTHexapodID==2
dfh2 = dfh[idx2]

In [ ]:
fig, ax = plt.subplots(1,1, figsize=(15,4))
#plt.plot(dfm.actualPosition, '--', label='mount elevation')
plt.plot(dfm1m3.zForces0*1e3, label='M1M3 aos force 101 (x1000N)')
plt.plot(dfm2.applied0*1e2, label='M2 aos force B1 (x100N)')
plt.plot(dfh1.y, '*', label='Camera hexapod y (microns)')
plt.plot(dfh2.y, 'o', label='M2 hexapod y (microns)')
plt.grid()
plt.legend()

### Step 2: add same aberrations twice. Are the aggregateDOF accumulating?

In [ ]:
await aos.cmd_addAberration.set_start(wf = zernikes, config=config_yaml)

In [ ]:
await aos.cmd_issueCorrection.set_start() 

In [ ]:
ofc_dict2 = await ofcSentApplied(aos, m1m3, m2, camhex, m2hex, make_plot=True)

This depends on the control strategy we are using. For x00, do not expect 2 or 1.

In [ ]:
config_yaml

In [ ]:
ofc_dict['visit_dof'][2], ofc_dict2['visit_dof'][2]

In [ ]:
ofc_dict2['camhexC'][:5]/ofc_dict['camhexC'][:5]

In [ ]:
ofc_dict2['m2hexC'][:5]/ofc_dict['m2hexC'][:5]

In [ ]:
plt.plot(ofc_dict2['aggregated_dof'][:10]/ofc_dict['aggregated_dof'][:10])

In [ ]:
plt.plot(ofc_dict2['visit_dof'][:10]/ofc_dict['visit_dof'][:10])

### Step 3: Reset the corrections

In [ ]:
await aos.cmd_resetCorrection.set_start()
await aos.cmd_issueCorrection.set_start() 

In [ ]:
ofc_dict3 =  await ofcSentApplied(aos, m1m3, m2, camhex, m2hex, make_plot = True)

### Step 4: Double the aberration, do the corrections double? yes

In [ ]:
zernikes[7-4] = 2 # z7 = 2um

In [ ]:
await aos.cmd_addAberration.set_start(wf = zernikes) #config=??

In [ ]:
await aos.cmd_issueCorrection.set_start() 

In [ ]:
ofc_dict4 =  await ofcSentApplied(aos, m1m3, m2, camhex, m2hex, make_plot = True)

In [ ]:
ofc_dict4['camhexC'][:5]/ofc_dict['camhexC'][:5]

In [ ]:
ofc_dict4['m2hexC'][:5]/ofc_dict['m2hexC'][:5]

### Close up

In [ ]:
#if we started with enabled state, we need to put it back
await salobj.set_summary_state(m1m3, salobj.State.ENABLED,  settingsToApply='Default')

In [ ]:
await salobj.set_summary_state(m2, salobj.State.OFFLINE)

In [ ]:
await salobj.set_summary_state(m1m3, salobj.State.STANDBY)

In [ ]:
await salobj.set_summary_state(camhex, salobj.State.STANDBY)

In [ ]:
await salobj.set_summary_state(aos, salobj.State.STANDBY)

In [ ]:
await lowerM1M3(m1m3)

### Check if the telescope is in tracking mode. If yes, need to stop stacking. 
The alternative is to check "MT Mount status" dash board on Chronograf. Make sure there are three "False".

In [ ]:
mountStatus = await mount.evt_axesInPosition.aget(timeout=5.)
rotStatus = await rot.evt_inPosition.aget(timeout=5.)
trackingStatus = mountStatus.elevation and mountStatus.azimuth and rotStatus.inPosition
print('Are we tracking?', trackingStatus)

In [ ]:
await ptg.cmd_stopTracking.set_start(timeout=5.)